# Project 2: Thermoelectric Generator

## Import and Install Libraries

In [15]:
%%capture
%pip install pandas pyfluids sympy scipy numpy matplotlib openpyxl

import pandas as pd # This makes nice tables
import pyfluids # This does the thermodynamics
import math
import numpy as np
from scipy.optimize import fsolve
from scipy.optimize import root
import matplotlib.pyplot as plt

## Power Requirements

In [31]:
power_data = [
    ["Raspberry Pi Pico", 42, 0.1386],
    ["Lora Radio Module", 125, 0.4125],
    ["pH Sensor", 13.3, 0.04389],
    ["Salinity", 34, 0.1122],
]
power_draw = pd.DataFrame(power_data, columns=["Component", "Peak Amperage Draw (mA)", "Power Draw (W)"])

total_power_draw = power_draw['Power Draw (W)'].sum()

## Temperature Data

In [17]:
minT_data = [
    [1,24],
    [2,25],
    [3,26],
    [4,21],
]
min_Temp = pd.DataFrame(minT_data, columns=["Month", "Min Average Temp (C)"])

min_Temp

,Month,Min Average Temp (C)
0,1,24
1,2,25
2,3,26
3,4,21


In [18]:
maxT_data = [
    [1,32],
    [2,36],
    [3,37],
    [4,32],
]
max_Temp = pd.DataFrame(maxT_data, columns=["Month", "Max Average Temp (C)"])

max_Temp

,Month,Max Average Temp (C)
0,1,32
1,2,36
2,3,37
3,4,32


## Solar Irradiation

In [19]:
irradiation_data = [
    ["Min",4.38],
    ["Max",4.93],
]
irradiation = pd.DataFrame(irradiation_data, columns=["Solar Irradiation", "Daily Output (kWh/m^2)"])

irradiation

,Solar Irradiation,Daily Output (kWh/m^2)
0,Min,4.38
1,Max,4.93


## Wind Speed

In [20]:
wind_Data = [[3]]
wind_Speed = pd.DataFrame(wind_Data,columns = ["Average Wind Speed (m/s)"])
wind_Speed

,Average Wind Speed (m/s)
0,3


## Hourly Constants

In [21]:
# Load the Excel file into a pandas DataFrame
constant_frame = pd.read_excel('HourlyConstants.xlsx', header=1)

print(constant_frame.columns)

# Now you can work with the DataFrame 'df'
print(constant_frame.head())  # Display the first few rows of the DataFrame?

Index(['Month', 'Day', 'Hour', 'Beam Irradiance (W/m2)',
       'Diffuse Irradiance (W/m2)', 'Ambient Temperature (C)',
       'Wind Speed (m/s)'],
      dtype='object')
   Month  Day  Hour  Beam Irradiance (W/m2)  Diffuse Irradiance (W/m2)  \
0      6    1     0                       0                          0   
1      6    1     1                       0                          0   
2      6    1     2                       0                          0   
3      6    1     3                       0                          0   
4      6    1     4                       0                          0   

   Ambient Temperature (C)  Wind Speed (m/s)  
0                       21               0.8  
1                       21               0.8  
2                       20               0.7  
3                       20               0.7  
4                       20               0.7  


## Heat Transfer Characteristics

In [22]:
heat_data = [
    ["Number of Units",21,"3 sets of 7 in series"],
    ["Unit Length",0.03,"m"],
    ["Unit Thickness",3.75E-3,"m"],
    ["Thermal Conductivity",1.72,"W/mK"],
    ["Emissivity",.9,"Painted Black"],
    ["Absorptivity",.9,"Painted Black"],
    ["Assumed Water Temperature",30,"Degree C"],
]
heat_parameters = pd.DataFrame(heat_data, columns=["Parameter", "Value","Units/Notes"])

heat_parameters

,Parameter,Value,Units/Notes
0,Number of Units,21.00000,3 sets of 7 in series
1,Unit Length,0.03000,m
2,Unit Thickness,0.00375,m
3,Thermal Conductivity,1.72000,W/mK
4,Emissivity,0.90000,Painted Black
5,Absorptivity,0.90000,Painted Black
6,Assumed Water Temperature,30.00000,Degree C


In [23]:
from pyfluids import Fluid, FluidsList, Input

water = Fluid(FluidsList.Water)

water_ambient = water.with_state(
    Input.pressure(101325), Input.temperature(30)
)
#print(water_ambient.density)

## Natural Convection for the Bottom of the Plate

In [24]:
def calcBeta(Tc,Tw):
    """
    Calculates Beta coefficient

    Parameters:
    -Tc (float): Temp (K) of the cold side of the thermoelectric generator
    -Th (float): Temp (K) of the hot side of the thermoelectric generator

    Returns:
    -beta (float): coefficient value
    """
    water_Amb = water.with_state(
    Input.pressure(101325), Input.temperature(Tw-273)
    )

    rhoAmb = water_Amb.density

    T = Tc - 273
    rho = (999.83952 + 16.945176 * T - 7.9870401E-3 * (T**2) - 49.170461E-6 * (T**3) + 105.56302E-9 * (T**4) - 280.54253E-12 * (T**5))/(1+16.897850E-3 * T)
    print(rho)

    beta = -1/rho * (rhoAmb-rho)/(Tw-Tc)
    return beta


In [25]:
def calcRalyeighNumber(beta, Tc, Tw, x, alpha):
    """
    Calculates RalyeighNumber

    Parameters:
    -Tc (float): Temp (K) of the cold side of the thermoelectric generator
    -beta (float): coefficient value
    -Tw (float): Temp (K) of the water
    -x (float): Length in meters
    -alpha (float)

    Returns:
    -RaX
    """
    #Find dT
    deltaT = Tc - Tw
    #Define Constants
    g = 9.81
    kinematic_viscosity = 8.005E-7
    #Calculate Ra
    Rax  = abs(g * beta * deltaT * (x)**3 / (kinematic_viscosity * alpha))
    return Rax

In [26]:
def bottomConvection(Tc,Tw,L,alpha):

    Pr = 6.9
    beta = calcBeta(Tc,Tw)
    
    #Calculate Ralyeigh Number at x = L
    RaL = calcRalyeighNumber(beta,Tc, Tw, L, alpha)

    assert RaL < 1e9 and RaL > 1e4, "Ralyeigh Number is out of bounds"
    
    NuL = 0.52 * RaL ** (1/5)

    kf = 0.598E-3 # kW/mK


    return NuL



## Forced Convection on Top of the Plate

In [27]:
def topForcedConvectionCoefficient(u,Lplate):
    """
    Calculates h for the top plate due to wind

    Parameters:
    -u (float): Wind speed (m/s)
    -Lplate (float):Length of solar abosrber (m)

    Returns:
    -ha (float): Convection coefficient (W/m^2 K)
    """
    #Forced Convection on the top of the plate
    #Constants for 
    kf = 27.06 #W/(mk)
    kinematic_viscosity = 1.61E-5
    Pr = 0.71

    Re = u  * Lplate / (kinematic_viscosity)
    
    assert Re < 1e6, "Reynolds Number Exceeded"

    NuL = 0.453 * (Re ** (1/2)) * (Pr ** 1/3)
    ha = NuL * kf / Lplate

    return ha


## Heat Balance Equations

In [28]:
def updatedSolution(u, L_plate, T_algae, L, alpha, G, epsilon, Tamb, Kt, N, t, L_sink):
    """
    Numerically solves for Th (Temp (K) of hot side of thermoelectric generator), and Tc (Temp (K) of cold side of thermoelectric generator)

    Parameters:
    -u (float): Wind speed (m/s)
    -L_plate (float):Length of solar abosrber (m)
    -L (float): length of thermoelectric generator module (m)
    -alpha (float):
    -G (float): 
    -epsilon (float): emmisivity 
    -Tamb (float): Ambient Temperature (K)
    -Kt (float): Thermal conductivity of the thermoelectric element
    -N (int): number of modules
    -t (float): Thickness of the thermoelectric element (m)
    -L_sink (float): Length of the heat sink (m)

    
    """    

    def residuals(vars):
        Th, Tc, h_w, rho = vars

        ha = topForcedConvectionCoefficient(u, L_plate)
        stefan = 5.67E-8

        top_plate = L_plate * L_plate * (G * alpha - stefan * epsilon * Th ** 4 - ha * (Th - Tamb))
        through_gen = N * L * L * Kt * t * (Th - Tc)
        into_water = h_w * (Tc - T_algae) * L_sink * L_sink

        top_to_gen = top_plate - through_gen
        gen_to_water = through_gen - into_water

        # Calculate rho
        water_Amb = water.with_state(Input.pressure(101325), Input.temperature(T_algae-273))
        rhoAmb = water_Amb.density
        rho_expr = (999.83952 + 16.945176 * (Tc - 273) - 7.9870401E-3 * ((Tc - 273)**2) - 49.170461E-6 * ((Tc - 273)**3) + 105.56302E-9 * ((Tc - 273)**4) - 280.54253E-12 * ((Tc - 273)**5))/(1+16.897850E-3 * (Tc - 273))
        rho_balance = rho - rho_expr

        Beta = -1/rho * (rhoAmb-rho)/(T_algae - Tc)

        # Calculate RaL
        deltaT = Tc - T_algae
        g = 9.81
        kinematic_viscosity = 8.005E-7
        RaL = abs(g * Beta * deltaT * (L_sink)**3 / (kinematic_viscosity * alpha))

        Pr = 6.9
        NuL = 0.68 + 0.670 * (RaL) ** (1/4) * (1 - (0.492/Pr) ** (9/16)) ** (-4/9)
        
        water_convective_coeff = h_w - NuL * 0.6 / L
        return [top_to_gen, gen_to_water, water_convective_coeff, rho_balance]

    initial_guess = [400, 300, 50, 900]
    s = root(residuals, initial_guess, method = 'lm')

    return s

sol = updatedSolution(u = 0.5, L_plate = 1, T_algae = 30 + 273.15, L = 30E-3, alpha = 0.9, G = 800, epsilon = 0.9, Tamb = 36 + 273.15, Kt = 1.7e-3, N = 21, t = 3e-3, L_sink = 0.25)
print(sol)

print("T_h (˚C):", sol.x[0] - 273.15)
print("T_c (˚C):", sol.x[1] - 273.15)
print("delta T:", sol.x[0] - sol.x[1])
print("h_w:", sol.x[2])
print("rho:", sol.x[3])


 message: The relative error between two consecutive iterates is at most 0.000000
 success: True
  status: 2
     fun: [-2.490e-12  3.329e-14 -2.515e-12  1.137e-13]
       x: [ 3.096e+02  3.032e+02  5.968e+01  9.952e+02]
   cov_x: [[ 3.737e-06  1.349e-11  1.184e-10 -4.259e-12]
           [ 1.349e-11  7.187e-02  6.309e-01 -2.269e-02]
           [ 1.184e-10  6.309e-01  7.794e+02 -2.800e+01]
           [-4.259e-12 -2.269e-02 -2.800e+01  1.007e+00]]
  method: lm
    nfev: 36
    fjac: [[ 5.173e+02 -1.863e-10 -0.000e+00 -0.000e+00]
           [ 0.000e+00 -2.782e+01  9.994e-01  3.595e-02]
           [-9.709e-08 -1.135e-02 -3.744e+00  1.201e-01]
           [-2.237e-18 -9.994e-01  3.030e-03  3.582e-02]]
    ipvt: [1 4 2 3]
     qtf: [ 2.487e-12 -1.419e-07 -9.030e-08  1.242e-08]
T_h (˚C): 36.49077130305301
T_c (˚C): 30.000000167720998
delta T: 6.490771135332011
h_w: 59.684395297413644
rho: 995.1894532549267
